# Fine-tuning a model on a translation task

In this notebook, we will see how to fine-tune one of the hugging-face Transformers model for a english to swahili translation task.The model we will be fine tuning is Helsinki-NLP/opus-mt-en-sw which was trained by University of Helsinski in Germany <br>
Below is the example translation text using pre-trained model
Reference : https://huggingface.co/Helsinki-NLP/opus-mt-en-sw


# Install required packages

---



In [17]:
!pip install -q sacremoses #tokenization and normalization tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [18]:
!pip install -q sacrebleu #evualuation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 4.6 MB/s eta 0:00:00


In [19]:
!pip install -q evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 29.9 MB/s eta 0:00:00


In [20]:
!pip install -q  sentencepiece transformers[sentencepiece]  #tokenizer and detokenizer 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.


Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version

In [21]:
import transformers
print(transformers.__version__)

4.28.1


In [4]:
import pandas as pd
import re #regular expression
import string
import warnings
warnings.filterwarnings("ignore")


# Import the datasets

In [2]:
from google.colab import drive

drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
rows = 8000 #take the first 8000 thousand rows
df = pd.read_csv('/content/gdrive/MyDrive/combined.csv' ,index_col=False)

In [6]:
#visualize the top 10 datasets 
df.head()

,English sentence,Swahii Translation
0,The government has constructed better health c...,Serikali imejenga vituo bora vya afya vijijini.
1,His death was so sudden.,Kifo chake kilikuwa ghafla.
2,Our company celebrated twenty years of existin...,Kampuni yetu ilisherehekea miaka ishirini ya ...
3,Many people have lost their lives due to sexua...,Watu wengi wamepoteza maisha yao kwa sababu y...
4,Civil servants receive their pension fund upon...,Wafanyikazi wa umma wanapokea fedha zao za pe...


In [7]:
# converting every letter to lower case
df['English sentence'] = df['English sentence'].apply(lambda x: str(x).lower())
df['Swahii Translation'] = df['Swahii Translation'].apply(lambda x: str(x).lower())

In [8]:
# removing apostrophe from the sentences
df['English sentence'] = df['English sentence'].apply(lambda x: re.sub("'","",x))
df['Swahii Translation'] = df['Swahii Translation'].apply(lambda x: re.sub("'","",x))

In [9]:
# removing digits from the sentences
digit = str.maketrans('','',string.digits)
df['English sentence'] = df['English sentence'].apply(lambda x: x.translate(digit))
df['Swahii Translation'] = df['Swahii Translation'].apply(lambda x: x.translate(digit))

In [10]:
#cleaned datset
df.head()

,English sentence,Swahii Translation
0,the government has constructed better health c...,serikali imejenga vituo bora vya afya vijijini.
1,his death was so sudden.,kifo chake kilikuwa ghafla.
2,our company celebrated twenty years of existin...,kampuni yetu ilisherehekea miaka ishirini ya ...
3,many people have lost their lives due to sexua...,watu wengi wamepoteza maisha yao kwa sababu y...
4,civil servants receive their pension fund upon...,wafanyikazi wa umma wanapokea fedha zao za pe...


In [11]:
df = df.set_index(df.index, drop=True)

In [12]:
#convert it back to csv
df = df.reset_index(drop=True)
df = df.to_csv("combined_processed", index=False)

# Loading the dataset

We will use the datasets library to load the already pre processed data from our local storage. 

In [13]:
from datasets import load_dataset

In [14]:
dataset = load_dataset("csv", data_files="/content/combined_processed")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-221df9d6fadd2b8a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

The dataset object itself is datasetdict, which contains only one split.We have 8492 pairs of sentences.

In [15]:
# a view on the dataset object
dataset

DatasetDict({
    train: Dataset({
        features: ['English sentence', 'Swahii Translation'],
        num_rows: 8492
    })
})

#Split dataset for training and modelling

We have 8492 pairs of sentences, but in one single split, so we will need to create our own validation set

In [16]:
split_datasets = dataset["train"].train_test_split(train_size=0.8, seed=20)
split_datasets

DatasetDict({
    train: Dataset({
        features: ['English sentence', 'Swahii Translation'],
        num_rows: 6793
    })
    test: Dataset({
        features: ['English sentence', 'Swahii Translation'],
        num_rows: 1699
    })
})

In [19]:
# a look on the dataset
split_datasets["train"][1]["English sentence"], split_datasets["train"][10]["Swahii Translation"]

('i cant let anyone watch what i am doing.', 'nililala usiku wa jana.')

# Loading pretrained model

In [20]:
from transformers import pipeline
# swahili pretraiined model from hugging face
model_checkpoint = "Helsinki-NLP/opus-mt-en-sw"
pretrained_translator = pipeline("translation", model=model_checkpoint)

In [22]:
# test pretrained model
pretrained_translator("i slept late last night")

[{'translation_text': 'Hula usiku sana'}]

# Pre processing with Tokenizers 

Before we can feed those texts to our model, we need to preprocess them. This is done by a Transformers Tokenizer which will (as the name indicates) tokenize the inputs and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the AutoTokenizer.from_pretrained method, which will ensure:

we get a tokenizer that corresponds to the model architecture we want to use, we download the vocabulary used when pretraining this specific checkpoint. 



In [23]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-en-sw"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

You can directly call this tokenizer on one sentence or a pair of sentences:

In [41]:
#encoder 
en_sentence = split_datasets["train"][1]["English sentence"]
sw_sentence = split_datasets["train"][1]["Swahii Translation"]

inputs = tokenizer(en_sentence, text_target=sw_sentence)
inputs

{'input_ids': [533, 77, 209, 575, 1542, 3259, 84, 533, 477, 814, 3, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [8219, 20182, 122, 791, 13, 12062, 1133, 23, 33840, 3, 0]}

In [42]:
#decoder
print(tokenizer.convert_ids_to_tokens(inputs["labels"]))

['▁siwezi', '▁kumruhusu', '▁mtu', '▁yeyote', '▁a', 'angalie', '▁kile', '▁ni', 'nachofanya', '.', '</s>']


We can then write the function that will preprocess our samples. We just feed them to the tokenizer with the argument truncation=True. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. 
Note that we set the same maximum length for our inputs and outputs. Since the texts we’re dealing with seem pretty short, we use 128.

In [24]:
max_length = 128

def preprocess_function(examples):
    inputs = examples['English sentence']
    targets = examples['Swahii Translation']
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [44]:
preprocess_function(split_datasets['train'][:2])

{'input_ids': [[4, 2911, 2399, 4, 7287, 17701, 17, 3, 0], [533, 77, 209, 575, 1542, 3259, 84, 533, 477, 814, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[2518, 8427, 8550, 10, 29236, 10, 5842, 3, 0], [8219, 20182, 122, 791, 13, 12062, 1133, 23, 33840, 3, 0]]}

To apply this function on all the pairs of sentences in our dataset, we just use the map method of our dataset object we created earlier. This will apply the function on all the elements of all the splits in dataset, so our training and validation data will be preprocessed in one single command

In [25]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True
)

Map:   0%|          | 0/6793 [00:00<?, ? examples/s]

Map:   0%|          | 0/1699 [00:00<?, ? examples/s]

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the AutoModelForSeq2SeqLM class. Like with the tokenizer, the from_pretrained method will download and cache the model for us.

In [26]:
from transformers import TFAutoModelForSeq2SeqLM,  Seq2SeqTrainingArguments,Seq2SeqTrainer

model = TFAutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-sw", from_pt=True)

All PyTorch model weights were used when initializing TFMarianMTModel.

All the weights of TFMarianMTModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


### Data collation

We can now use this data_collator to convert each of our datasets to a tf.data.Dataset, ready for training:

In [27]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [28]:
tf_train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=32,
)
tf_eval_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

# Setting up Sacrebeu for evaluation

The BLEU score evaluates how close the translations are to their labels. It does not measure the intelligibility or grammatical correctness of the model’s generated outputs, but uses statistical rules to ensure that all the words in the generated outputs also appear in the targets.One weakness with BLEU is that it expects the text to already be tokenized, which makes it difficult to compare scores between models that use different tokenizers. So instead, the most commonly used metric for benchmarking translation models today is SacreBLEU, which addresses this weakness  by standardizing the tokenization step.
The score can go from 0 to 100, and higher is better.

In [29]:
import evaluate

metric = evaluate.load("sacrebleu")

In [30]:
# testing  sacrabeu on correct prediction
predictions = [
    "Tutaunda ukuta mpya kuzunguka shule yote."
]
references = [
    [
        "Tutatengeneza ukuta mpya kuzunguka shule."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 43.47208719449914,
 'counts': [5, 3, 2, 1],
 'totals': [7, 6, 5, 4],
 'precisions': [71.42857142857143, 50.0, 40.0, 25.0],
 'bp': 1.0,
 'sys_len': 7,
 'ref_len': 6}

In [51]:
# testing  on incorrrect  translation
predictions = [
    "Today is a very beatiful day "
]
references = [
    [
        "Python is a language mostly used for machine learning"
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 9.846052248031867,
 'counts': [2, 1, 0, 0],
 'totals': [6, 5, 4, 3],
 'precisions': [33.333333333333336, 20.0, 12.5, 8.333333333333334],
 'bp': 0.6065306597126334,
 'sys_len': 6,
 'ref_len': 9}

### Function to loop through all translation and give evaluation

 Let’s define a function that takes our model and a dataset and computes metrics on it. We’re also going to use a trick that dramatically increases performance - compiling our generation code with XLA, TensorFlow’s accelerated linear algebra compiler. XLA applies various optimizations to the model’s computation graph, and results in significant improvements to speed and memory usage

In [31]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm

generation_data_collator = DataCollatorForSeq2Seq(
    tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128
)

tf_generate_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    collate_fn=generation_data_collator,
    shuffle=False,
    batch_size=8,
)


@tf.function(jit_compile=True)
def generate_with_xla(batch):
    return model.generate(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        max_new_tokens=128,
    )


def compute_metrics():
    all_preds = []
    all_labels = []

    for batch, labels in tqdm(tf_generate_dataset):
        predictions = generate_with_xla(batch)
         # Replace -100 in the labels as we can't decode them.
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        labels = labels.numpy()
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        # Some simple post-processing
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        decoded_preds = [pred.strip() for pred in decoded_preds]
        decoded_labels = [[label.strip()] for label in decoded_labels]
        all_preds.extend(decoded_preds)
        all_labels.extend(decoded_labels)

    result = metric.compute(predictions=all_preds, references=all_labels)
    return {"bleu": result["score"]}

### Score of pretrained model before fine tuning

Before we fine tuning, let’s see what kind of results we get from the  pretrained model without any training

In [53]:
print(compute_metrics())

100%|██████████| 213/213 [02:21<00:00,  1.50it/s]

{'bleu': 31.410094811457085}


# Fine tuning the pretrained model

We can now finetune our model by just calling the compile method

In [32]:
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

num_epochs = 2
num_train_steps = len(tf_train_dataset) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=5e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,

)
model.compile(optimizer=optimizer)


No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


### Save the model to huggingface

The first step is to log in to Hugging Face, so you’re able to upload your results to the Model Hub. There’s a convenience function to help you with this in a notebook.<br>
This will display a widget where you can enter your Hugging Face login credentials, paste the token from your Hugging Face account.

In [33]:
# login to hugging face 
from huggingface_hub import notebook_login
notebook_login()

Define a PushToHubCallback to upload our model to the Hub during training. Indicate the name of your model in the output dir

In [35]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir="final_eng-sw_translation", tokenizer=tokenizer
)

model.fit(
    tf_train_dataset,
    validation_data=tf_eval_dataset,
    callbacks=[callback],
    epochs=num_epochs,
)

/content/final_eng-sw_translation is already a clone of https://huggingface.co/KigenCHESS/final_eng-sw_translation. Make sure you pull the latest changes with `repo.git_pull()`.


Epoch 1/2
212/212 [==============================] - 120s 405ms/step - loss: 0.9548 - val_loss: 0.7129
Epoch 2/2
212/212 [==============================] - 80s 376ms/step - loss: 0.5624 - val_loss: 0.6716


### Model accuracy after fine tuning

Finally, let’s check what our metrics look like now that training has finished:

In [37]:
print(compute_metrics())

100%|██████████| 213/213 [02:21<00:00,  1.50it/s]


{'bleu': 59.722095402260265}


# Loading and using the pretrained model

In [38]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "final_eng-sw_translation"
fine_tuned_model = pipeline("translation", model=model_checkpoint)


All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at final_eng-sw_translation.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


### translation on pretrained model

In [40]:
fine_tuned_model("i slept late last night")

[{'translation_text': 'nililala jana usiku'}]

In [41]:
fine_tuned_model(" i will not go to school today")

[{'translation_text': 'sitaenda shule leo'}]

In [42]:
pretrained_translator(" i will not go to school today")

[{'translation_text': 'Siwezi kwenda shuleni leo'}]

# Comparing translations from di
### input text : i will not go to school today
### pretrained model prediction : Siwezi kwenda shuleni leo
### fine tuned model prediction : Sitaenda shule leo
### google translate prediction : sitaenda shule leo

# Deployment Of the Model

There are several ways to deploy a machine learning model including flask, Streamlit and others.For now we have deployed using gradio which is a python ui for proving interfaces for machine learning projects using short lines of code

In [44]:
!pip install -q gradio 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.2/286.2 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.5/140.5 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.7/129.7 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.6 MB/s eta 0:0

In [45]:
import gradio as gr

Create a function to generate the output text from the translation pipeline

In [46]:
def translate_transformers(from_text):
    results = fine_tuned_model(from_text)
    return results[0]['translation_text']

Pass the function to gradio interface. You can cahnge the place holder to suite your own preferences

In [48]:
interface = gr.Interface(fn=translate_transformers, 
                         inputs=gr.inputs.Textbox(placeholder='English Text to translate'),
                        outputs='text')

Finnally launch the model. You can use here or click on the link to you use on your browser window

In [50]:
interface.launch(share=True)


Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f4957d35741ae42614.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


# The END

## You have successfully fine-tuned and deployed a model on a translation task <br> CONGRACULATIONS!